In [2]:
import json
import time
import uuid
import random
from datetime import datetime
import pytz

In [3]:
import json
import time
import uuid
import random
from datetime import datetime
import pytz

# Definir las coordenadas aproximadas de Medellín
lat_range = (6.217, 6.317)  # Latitud de Medellín
lon_range = (-75.567, -75.467)  # Longitud de Medellín

# Lista de comunas y barrios de Medellín
comunas = ["COMUNA 1", "COMUNA 2", "COMUNA 3", "COMUNA 4", "COMUNA 5", "COMUNA 6", "COMUNA 7", "COMUNA 8", "COMUNA 9", "COMUNA 10", "COMUNA 11", "COMUNA 12", "COMUNA 13", "COMUNA 14", "COMUNA 15", "COMUNA 16"]
barrios = ["BARRIO 1", "BARRIO 2", "BARRIO 3", "BARRIO 4", "BARRIO 5", "BARRIO 6", "BARRIO 7", "BARRIO 8", "BARRIO 9", "BARRIO 10"]

# Generar datos simulados
def generate_event():
    event = {
        "latitude": round(random.uniform(*lat_range), 6),
        "longitude": round(random.uniform(*lon_range), 6),
        "date": datetime.now(pytz.timezone('America/Bogota')).strftime("%d/%m/%Y %H:%M:%S"),
        "customer_id": random.randint(1000, 2000),
        "employee_id": random.randint(9000, 10000),
        "quantity_products": random.randint(1, 50),
        "order_id": str(uuid.uuid4()),
        "commune": random.choice(comunas),
        "neighborhood": random.choice(barrios),
        "partition_date": datetime.now().strftime("%d%m%Y"),
        "event_date": datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
        "event_day": datetime.now().day,
        "event_hour": datetime.now().hour,
        "event_minute": datetime.now().minute,
        "event_month": datetime.now().month,
        "event_second": datetime.now().second,
        "event_year": datetime.now().year
    }
    return event

def main():
    while True:
        event = generate_event()
        # Aquí se puede guardar el evento en un archivo JSON, base de datos, etc.
        # Ejemplo de guardado en un archivo JSON
        with open('simulated_data.json', 'a') as file:
            file.write(json.dumps(event) + "\n")
        # Esperar 30 segundos antes de generar el siguiente evento
        time.sleep(30)

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [8]:
import pandas as pd

# Ruta al archivo Parquet
parquet_file = '../base.data/50001.parquet'

# Leer el archivo Parquet con pandas
df = pd.read_parquet(parquet_file)

# Mostrar las primeras filas del DataFrame
print(df.head())


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)